题目:
1. 使用pytorch搭建神经网络模型，实现对KMNIST数据集的训练。
https://pytorch.org/vision/stable/generated/torchvision.datasets.KMNIST.html#torchvision.datasets.KMNIST



In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001
# KMNIST dataset
train_dataset = torchvision.datasets.KMNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.KMNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Fully connected neural network with one hidden layer
# class NeuralNet(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(NeuralNet, self).__init__()
#         # 1st hidden layer: 784 inputs, 500 neurons
#         # 线性变换
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         # ReLU activation function
#         # 隐层激活函数
#         self.relu = nn.ReLU()
#         # output layer: 500 inputs, 10 neurons(输出层)
#         # 线性变换
#         self.fc2 = nn.Linear(hidden_size, num_classes)
#         # 多分类时, 输出层一般不使用激活函数, CrossEntropyLoss 内部已经实现了 Softmax, 因此不用激活函数
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out
# model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.Sigmoid(),
    nn.Linear(hidden_size, num_classes)
)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Adam 优化器, 效果比 SGD 好
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 把数据从 [100, 1, 28, 28] 转换为 [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device) 
        # Forward pass
        outputs = model(images) # 前向传播
        
        loss = criterion(outputs, labels) # 计算损失
        # Backward and optimize
        optimizer.zero_grad() # 清空梯度
        loss.backward() # 反向传播
        optimizer.step() # 更新参数
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# 测试模型, 不需要计算梯度
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


2. 尝试调整模型结构（变更神经元数量，增加隐藏层）来提升模型预测的准确率

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 超参数
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 数据预处理（添加标准化）
transform = transforms.Compose([
    transforms.ToTensor(), # 1. 将图像转为Tensor，并归一化到[0,1]
    transforms.Normalize((0.5,), (0.5,))   # 2. 标准化到[-1,1], 即均值为0.5, 方差为0.5, 好处是收敛更快
])

# KMNIST数据集
train_dataset = torchvision.datasets.KMNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.KMNIST(
    root='./data', train=False, transform=transform)

# 数据加载器
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 模型定义（使用ReLU激活）
hidden_1_size = 512
hidden_2_size = 128
model = nn.Sequential(
    nn.Linear(input_size, hidden_1_size), # 第一个隐藏层线性变换
    nn.ReLU(), # 第一个隐藏层激活函数
    nn.Linear(hidden_1_size, hidden_2_size), # 第二个隐藏层线性变换
    nn.ReLU(), # 第二个隐藏层激活函数
    nn.Linear(hidden_2_size, num_classes), # 输出层线性变换
).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # 建议使用Adam

# 训练循环
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# 测试
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/5], Step [100/600], Loss: 0.4987
Epoch [1/5], Step [200/600], Loss: 0.5436
Epoch [1/5], Step [300/600], Loss: 0.4735
Epoch [1/5], Step [400/600], Loss: 0.3390
Epoch [1/5], Step [500/600], Loss: 0.2770
Epoch [1/5], Step [600/600], Loss: 0.2713
Epoch [2/5], Step [100/600], Loss: 0.2250
Epoch [2/5], Step [200/600], Loss: 0.1808
Epoch [2/5], Step [300/600], Loss: 0.1324
Epoch [2/5], Step [400/600], Loss: 0.2197
Epoch [2/5], Step [500/600], Loss: 0.2122
Epoch [2/5], Step [600/600], Loss: 0.1509
Epoch [3/5], Step [100/600], Loss: 0.1110
Epoch [3/5], Step [200/600], Loss: 0.0748
Epoch [3/5], Step [300/600], Loss: 0.1950
Epoch [3/5], Step [400/600], Loss: 0.0916
Epoch [3/5], Step [500/600], Loss: 0.1062
Epoch [3/5], Step [600/600], Loss: 0.0995
Epoch [4/5], Step [100/600], Loss: 0.0952
Epoch [4/5], Step [200/600], Loss: 0.0621
Epoch [4/5], Step [300/600], Loss: 0.1100
Epoch [4/5], Step [400/600], Loss: 0.0561
Epoch [4/5], Step [500/600], Loss: 0.0617
Epoch [4/5], Step [600/600], Loss:

3. 调试超参数，观察学习率和批次大小对训练的影响。

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 超参数
input_size = 784
num_classes = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.002

# 数据预处理（添加标准化）
transform = transforms.Compose([
    transforms.ToTensor(), # 1. 将图像转为Tensor，并归一化到[0,1]
    transforms.Normalize((0.5,), (0.5,))   # 2. 标准化到[-1,1], 即均值为0.5, 方差为0.5, 好处是收敛更快
])

# KMNIST数据集
train_dataset = torchvision.datasets.KMNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.KMNIST(
    root='./data', train=False, transform=transform)

# 数据加载器
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 模型定义（使用ReLU激活）
hidden_1_size = 512
hidden_2_size = 128
model = nn.Sequential(
    nn.Linear(input_size, hidden_1_size),
    nn.ReLU(),
    nn.Linear(hidden_1_size, hidden_2_size),
    nn.ReLU(),
    nn.Linear(hidden_2_size, num_classes),
).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # 建议使用Adam

# 训练循环
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# 测试
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/10], Step [100/938], Loss: 0.4365
Epoch [1/10], Step [200/938], Loss: 0.6254
Epoch [1/10], Step [300/938], Loss: 0.2015
Epoch [1/10], Step [400/938], Loss: 0.3195
Epoch [1/10], Step [500/938], Loss: 0.4112
Epoch [1/10], Step [600/938], Loss: 0.3980
Epoch [1/10], Step [700/938], Loss: 0.2178
Epoch [1/10], Step [800/938], Loss: 0.2275
Epoch [1/10], Step [900/938], Loss: 0.1460
Epoch [2/10], Step [100/938], Loss: 0.1374
Epoch [2/10], Step [200/938], Loss: 0.1050
Epoch [2/10], Step [300/938], Loss: 0.1784
Epoch [2/10], Step [400/938], Loss: 0.2132
Epoch [2/10], Step [500/938], Loss: 0.2271
Epoch [2/10], Step [600/938], Loss: 0.1771
Epoch [2/10], Step [700/938], Loss: 0.1663
Epoch [2/10], Step [800/938], Loss: 0.1072
Epoch [2/10], Step [900/938], Loss: 0.2174
Epoch [3/10], Step [100/938], Loss: 0.0823
Epoch [3/10], Step [200/938], Loss: 0.1713
Epoch [3/10], Step [300/938], Loss: 0.1478
Epoch [3/10], Step [400/938], Loss: 0.0652
Epoch [3/10], Step [500/938], Loss: 0.0821
Epoch [3/10